# SOFTWARE PARA ARCHIVOS COMPRIMIDOS

In [ ]:
import openpyxl
import csv
import zipfile
import io

def obtener_registros_desde_zip(archivo_zip, hoja, coordenadas, nombre_archivo_salida):
    try:
        registros_globales = []  # Almacenará todos los registros de varios archivos Excel

        with zipfile.ZipFile(archivo_zip, 'r') as zip_file:
            with open(nombre_archivo_salida, 'w', newline='') as archivo_csv:
                writer = csv.writer(archivo_csv)

                for file_name in zip_file.namelist():
                    if file_name.endswith('.xlsx'):
                        excel_file = zip_file.read(file_name)
                        excel_data = io.BytesIO(excel_file)

                        # Abre el archivo Excel desde los datos del archivo zip
                        workbook = openpyxl.load_workbook(excel_data)
                        sheet = workbook[hoja]

                        registros = []

                        for coordenada in coordenadas:
                            # Divide la coordenada en fila y columna
                            fila, columna = openpyxl.utils.cell.coordinate_to_tuple(coordenada)

                            # Obtiene la celda actual
                            celda_actual = sheet.cell(row=fila, column=columna)

                            # Si la celda está combinada, encuentra la celda "inicial" dentro de la combinación
                            for rango_celdas_combinadas in sheet.merged_cells.ranges:
                                if celda_actual.coordinate in rango_celdas_combinadas:
                                    celda_actual = rango_celdas_combinadas.start_cell
                                    break

                            # Agrega el valor de la celda al registro actual
                            valor_celda = celda_actual.value
                            if isinstance(valor_celda, (int, float)):
                                registros.append(str(valor_celda))
                            elif valor_celda is not None:
                                registros.append(str(valor_celda))
                            else:
                                registros.append("")  # Valor nulo o vacío

                        writer.writerow(registros)  # Escribe los registros en el archivo CSV

        return nombre_archivo_salida
    except Exception as e:
        return f"Error: {e}"

# Uso de la función para obtener registros desde un archivo zip y guardar en un solo archivo CSV
archivo_zip = '/content/certificados_comprimidos.zip'
hoja = 'Certificado'
coordenadas = ['H15', 'A15', 'A9', 'B19', 'G19', 'H19', 'I19', 'C26', 'C28', 'C30', 'C34', 'C36', 'C38', 'C42', 'C44', 'C46', 'C48', 'C50', 'C52', 'H38', 'H46', 'H50', 'H52']
nombre_archivo_salida = '02_registros_globales.csv'
resultado = obtener_registros_desde_zip(archivo_zip, hoja, coordenadas, nombre_archivo_salida)

if resultado != nombre_archivo_salida:
    print(f"Error: {resultado}")
else:
    print(f"Registros listos para descargar en {nombre_archivo_salida}")



Registros listos para descargar en 02_registros_globales.csv
